In [1]:
!pip install caer canaro

In [8]:
import os
import caer
import numpy as np
import canaro
import cv2 as cv
import gc

In [9]:
IMG_SIZE = (80, 80)
channels = 1
char_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset'

In [11]:
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path, char)))
    
# Sort in descending order
char_dict = caer.sort_dict(char_dict, descending=True)
char_dict